In [28]:
%%capture
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
from IPython.display import display
from ipywidgets import HBox, VBox
from collections import OrderedDict
from fuzzywuzzy import process 
from fuzzywuzzy import fuzz
pd.options.display.html.table_schema = True
pd.options.display.max_rows = None
pd.options.display.max_columns = 10
pd.options.display.max_colwidth = 90
pd.options.display.width = 270
pd.options.display.expand_frame_repr = True
#import google.cloud.automl_v1beta1 as automl_v1beta1
from google.cloud import automl_v1beta1
from google.cloud.automl_v1beta1.proto import service_pb2
import textlib
from textlib import load_raw
from textlib import normalize_sent
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from io import StringIO




<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-H&amp;K-Lawyer-Notebook" data-toc-modified-id="The-H&amp;K-Lawyer-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The H&amp;K Lawyer Notebook</a></span><ul class="toc-item"><li><span><a href="#Classification-Tool" data-toc-modified-id="Classification-Tool-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Classification Tool</a></span></li><li><span><a href="#Bulk-Compare" data-toc-modified-id="Bulk-Compare-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Bulk Compare</a></span></li><li><span><a href="#Find-Precedent" data-toc-modified-id="Find-Precedent-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Find Precedent</a></span></li><li><span><a href="#Clustering-and-Text-Similarity-Models" data-toc-modified-id="Clustering-and-Text-Similarity-Models-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Clustering and Text Similarity Models</a></span></li><li><span><a href="#QuickCAN" data-toc-modified-id="QuickCAN-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>QuickCAN</a></span></li><li><span><a href="#QuickCAR" data-toc-modified-id="QuickCAR-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>QuickCAR</a></span></li><li><span><a href="#Signature-Page-Generator" data-toc-modified-id="Signature-Page-Generator-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Signature Page Generator</a></span></li><li><span><a href="#Closing-Binder-Generator" data-toc-modified-id="Closing-Binder-Generator-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Closing Binder Generator</a></span></li></ul></li></ul></div>

In [29]:
import csv
import os
import os.path
import re

from bs4 import BeautifulSoup as BeautifulSoup
from openpyxl import Workbook as Workbook

def save_list_as_excel(path, clean_sent_list, raw_sent_list):
    wb = Workbook()
    ws = wb.active
    column_cell_A = 'A'
    column_cell_B = 'B'
    column_cell_C = 'C'
    ws[column_cell_A + str(1)] = 'text'
    ws[column_cell_B + str(1)] = 'label'
    ws[column_cell_C + str(1)] = 'raw'
    list_len = len(clean_sent_list)
    for i in range(0, list_len):
        ws[column_cell_A + str(i + 2)] = clean_sent_list[i]
        ws[column_cell_B + str(i + 2)] = 'label'
        ws[column_cell_C + str(i + 2)] = raw_sent_list[i]
    wb.save(path)
    print('Excel write complete')


def html_from_file_no_tags(file_path):
    with open(file_path, 'rb') as myfile:
        raw_text = myfile.read()
    bsObj = BeautifulSoup(raw_text, 'lxml').text
    return bsObj


def normalize_sent(sent):
    wpt = nltk.WordPunctTokenizer()
    stop_words = nltk.corpus.stopwords.words('english')
    sent = re.sub(r'[^a-zA-Z\s]', '', sent, re.I | re.A)
    sent = sent.strip()
    tokens = wpt.tokenize(sent)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    sent = ' '.join(filtered_tokens).lower()
    return sent


def html_from_file_no_tags(file_path):
    with open(file_path, 'rb') as myfile:
        raw_text = myfile.read()
    bsObj = BeautifulSoup(raw_text, 'lxml').text
    return bsObj


def load_normal_with_stopwords(path):
    doc = str(load_raw(path))
    return normalize_document_return_list(doc)


def load_normal_no_stopwords(path):
    doc = load_raw(path)
    norm = normalize_document_return_list(doc)
    clean_sent = []
    for sent in norm:
        clean = remove_stop_words(sent)
        clean_sent.append(clean)
    return clean_sent


def list_from_directory(path):
    list_of_text = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        file_path = path + filename
        text = load_raw(file_path)
        text = str(text)
        list_of_text.append(text)


def load_excel(path):
    print('finish')


def load_list_from_csv(path):
    with open(path, 'r') as f:
        reader = csv.reader(f)
        list_raw = list(reader)
        list_clean = []
        for i in list_raw:
            x = ''.join(i)
            list_clean.append(x)
        list_clean_two = []
        for i in list_clean:
            x = ''.join(i)
            list_clean_two.append(x)
        return list_clean_two


def load_raw(path, tags=False):
    if path.endswith('html'):
        if (tags):
            return html_from_file_tags(path)
        else:
            return html_from_file_no_tags(path)
    elif path.endswith('.txt'):
        return str(text_from_file(path))
    else:
        try:
            return str(text_from_binary(path))
        except:
            print(
                'Failed to load as binary. Try reader that accepts url as argument (e.g., html_from_web_tags(url) or html_from_web_no_tags(url)).')


def text_from_binary(file_path):
    #text = textract.process(file_path, method='tesseract', language='eng')
    text = 'Not set up for binary'
    return text.decode('unicode_escape').encode('utf-8', 'ignore').strip()


def html_from_file_tags(file_path):
    with open(file_path, 'rb') as myfile:
        raw_text = myfile.read()
    return raw_text


def html_from_web_no_tags(url):
    response = urlopen(url)
    bsObj = BeautifulSoup(response, 'lxml').text
    return bsObj


def html_from_web_tags(url):
    response = urlopen(url)
    tagged_text = response.read()
    return tagged_text


def text_from_file(file_path):
    with open(file_path, 'rb') as myfile:
        raw_text = myfile.read()
    return raw_text


def remove_stop_words(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = nltk.corpus.stopwords.words('english')
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens).lower()
    return doc


def normalize_document_return_list(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = nltk.corpus.stopwords.words('english')
    list_of_clean_sents = []
    sent_list = tokenize.sent_tokenize(str(doc))
    for sent in sent_list:
        sent = re.sub(r'[^a-zA-Z\s]', '', sent, re.I | re.A)
        sent = sent.strip()
        tokens = wpt.tokenize(sent)
        filtered_tokens = [token for token in tokens if token not in stop_words]
        sent = ' '.join(filtered_tokens).lower()
        list_of_clean_sents.append(sent)
    return list_of_clean_sents


def tokenize_and_stem(text):
    stemmer = SnowballStemmer("english")
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    joined_text = " ".join(stems)
    print(joined_text)
    return joined_text


![alt text](https://s3.amazonaws.com/blaze4/HK+logo.png)
# The H&K Lawyer Notebook

A collection of simple, yet powerful tools for lawyers to do their jobs better and more efficiently. By leveraging data analytics, natural language processing, machine learning, network graphing and other open source solutions, users are able to produce better quality work faster and more efficiently. Many of these tools are still under active development, so the H&K Lawyer Notebook is reserved for our more intrepid lawyers.

***

## Classification Tool

Classify text using one of several trained machine learning models.

In [42]:
'''name = 'projects/ccapp-141701/locations/us-central1/models/TCN1260328743632064609'
prediction_client = automl_v1beta1.PredictionServiceClient.from_service_account_json('ccapp-141701-bf3fd9e2a4f7-demo.json')
def automl_predict(model_full_id, snippet):
    payload = {'text_snippet': {
        'content': snippet,
        "mime_type": "text/plain"
            }
      }
    return prediction_client.predict(model_full_id, payload)
'''
radio_buttons = widgets.RadioButtons(
    options=['finance', 'leasing', 'private equity', 'corporate M&A', 'bond financing'],
     value='finance',
    description='Select model:',
    disabled=False
)
display(radio_buttons)

clause_selection = widgets.SelectMultiple(
    options=['Termination', 'Commitment/Unused Fees', 'Change in Control', 'Voluntary Prepayment', 'Material Adverse Condition', 'Events of Default', 'Eurodollar Rate'],
    value=['Change in Control'],
    #rows=10,
    description='Select:',
    disabled=False
)
display(clause_selection)

file_path_text = widgets.Text(              
    value='credit.txt',
    placeholder='Type something',
    description='Filename:',
    disabled=False
)
display(file_path_text)

from IPython.display import display
button = widgets.Button(description="Submit")
button.style.button_color = 'lightblue'
display(button)

in_progress = widgets.IntProgress(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

def load_model():
    df = pd.read_excel('/home/ec2-user/SageMaker/data_b.xlsx')
    df = df.sample(frac=1, axis=1).reset_index(drop=True)
    df = df[pd.notnull(df['Text'])]
    col = ['Label', 'Text']
    df = df[col]
    df.columns = ['Label', 'Text']
    df = df.sample(frac=1).reset_index(drop=True)
    df['category_id'] = df['Label'].factorize()[0]
    category_id_df = df[['Label', 'category_id']].drop_duplicates().sort_values('category_id')
    category_to_id = dict(category_id_df.values)
    id_to_category = dict(category_id_df[['category_id', 'Label']].values)
    X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Label'], random_state = 0)
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    clf = LinearSVC().fit(X_train_tfidf, y_train)
    return clf

def on_button_clicked(b):
    in_progress.value = 2
    display(in_progress)
    df = pd.read_excel('/home/ec2-user/SageMaker/data_b.xlsx')
    df = df.sample(frac=1, axis=1).reset_index(drop=True)
    df = df[pd.notnull(df['Text'])]
    col = ['Label', 'Text']
    df = df[col]
    df.columns = ['Label', 'Text']
    df = df.sample(frac=1).reset_index(drop=True)
    df['category_id'] = df['Label'].factorize()[0]
    category_id_df = df[['Label', 'category_id']].drop_duplicates().sort_values('category_id')
    category_to_id = dict(category_id_df.values)
    id_to_category = dict(category_id_df[['category_id', 'Label']].values)
    X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Label'], random_state = 0)
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    clf = LinearSVC().fit(X_train_tfidf, y_train)
    in_progress.value = 4
    current_raw_sent_list = []
    current_clean_sent_list = []
    sent_list = sent_tokenize(load_raw(file_path_text.value))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        current_clean_sent_list.append(clean_sent)
        current_raw_sent_list.append(sent)
    in_progress.value = 6
    df_current = pd.DataFrame(columns=['label', 'raw', 'clean'])
    df_current['clean'] = current_clean_sent_list
    df_current['raw'] = current_raw_sent_list
    in_progress.value = 8
    for index, row in df_current.iterrows():
        test = []
        test.append(df_current['clean'][index])
        X_test=count_vect.transform(test)
        z = clf.predict(X_test)
        df_current['label'][index] = z[0]
        print('*********************************************************************************')
        print(row['clean'])
        print('is:')
        print(z[0])
        print('*********************************************************************************')
    in_progress.value = 10
    display(df_current)
    
button.on_click(on_button_clicked)

RadioButtons(description='Select model:', options=('finance', 'leasing', 'private equity', 'corporate M&A', 'b…

SelectMultiple(description='Select:', index=(2,), options=('Termination', 'Commitment/Unused Fees', 'Change in…

Text(value='credit.txt', description='Filename:', placeholder='Type something')

Button(description='Submit', style=ButtonStyle(button_color='lightblue'))

IntProgress(value=2, description='Loading:', max=10)

...

is:
voluntary_prepayment
*********************************************************************************
*********************************************************************************
during periods revolving loan abr borrowingnwhich shall times case swingline loan protectivenadvance rate per annum equal times sum a alternatenbase rate effect time time plus b applicable revolving loannmargin abr borrowings effect time timenn nniieurodollar loans
is:
voluntary_prepayment
*********************************************************************************
*********************************************************************************
during periods revolving loan aneurodollar borrowing rate per annum equal times interestnperiod loans sum a adjusted libo rate suchninterest period borrowing plus b applicable revolving loannmargin eurodollar loans effect time timenn nnb term loans
is:
eurodollar_rate
*********************************************************************************
*****

amendments material documents
is:
MAC_definition
*********************************************************************************
*********************************************************************************
neither holdings thenparent borrower permit subsidiary amend modify ornwaive rights certificate incorporation bylawsnoperating management partnership agreement organizationalndocuments b documents governing material indebtedness eachncase extent amendment modification waiver would benmaterially adverse lendersnn nnsection
is:
Amendments_Consent
*********************************************************************************
*********************************************************************************
minimum excess availability
is:
excess_cash_flow
*********************************************************************************
*********************************************************************************
loan parties shall maintain minimumnexcess availability times l

n n nn nn nn nn nn nn nnexhibit ann nnform ofnnassignment and assumptionnnthis assignment assumption xexxcassignment assumptionxexxd dated asnof effective date set forth entered betweenninsert name assignor xexxcassignorxexxd insert name assignee thenxexxcassigneexexxd
is:
margin_leverage_ratio
*********************************************************************************
*********************************************************************************
capitalized terms used defined herein shall thenmeanings given credit agreement identified amendednthe xexxccredit agreementxexxd receipt copy hereby acknowledged bynthe assignee
is:
Amendments_Consent
*********************************************************************************
*********************************************************************************
the standard terms conditions set forth annex attachednhereto hereby agreed incorporated herein reference made anpart assignment assumption set forth herein fullnnfor agreed 

each referencento xexxcloan partyxexxd xexxcsubsidiary guarantorxexxd xexxcguarantorxexxd thenn nn nn nn nncollateral agreement shall deemed include new subsidiary
is:
Amendments_Consent
*********************************************************************************
*********************************************************************************
thencollateral agreement hereby incorporated herein referencenn nnsection
is:
Amendments_Consent
*********************************************************************************
*********************************************************************************
the new subsidiary represents warrants administrativenagent secured parties supplement dulynauthorized executed delivered constitutes legal valid andnbinding obligation enforceable accordance termsnn nnsection
is:
secured_facility
*********************************************************************************
***************************************************************************

,label,raw,clean
0,secured_facility,b'EX-10.1 2 creditagreement.htm EXHIBIT Credit Agreement\n\n \n\n \n\n[Execution Ve...,bex creditagreementhtm exhibit credit agreementnn nn nnexecution versionnn nnpublished...
1,secured_facility,"C. PENNEY COMPANY, INC.,\n\nJ.",c penney company incnnj
2,secured_facility,"C. PENNEY CORPORATION, INC.,\n\nJ.",c penney corporation incnnj
3,lender_inspection_rights,"C. PENNEY PURCHASING CORPORATION,\n\n \n\nThe Lenders Party Hereto,\n\n \n\nWELLS FA...",c penney purchasing corporationnn nnthe lenders party heretonn nnwells fargo bank nati...
4,secured_facility,MORGAN SECURITIES LLC\n\nBARCLAYS BANK PLC\n\nGOLDMAN SACHS BANK USA\n\nas Term Joint ...,morgan securities llcnnbarclays bank plcnngoldman sachs bank usannas term joint bookru...
5,change_control,"MORGAN SECURITIES LLC\n\nBARCLAYS BANK PLC\n\nGOLDMAN SACHS BANK USA\n\nHSBC BANK USA,...",morgan securities llcnnbarclays bank plcnngoldman sachs bank usannhsbc bank usa nannci...
6,lender_inspection_rights,"MORGAN SECURITIES LLC,\n\nBARCLAYS BANK PLC\n\nGOLDMAN SACHS BANK USA\n\nHSBC BANK USA...",morgan securities llcnnbarclays bank plcnngoldman sachs bank usannhsbc bank usa nannci...
7,MAC_definition,Defined\nTerms...........................................................................,definedntermsnnsection classification loans andnborrowingsnnsection termsngenerally .....
8,lender_inspection_rights,]\n\nExhibit C -- Form of Collateral Agreement\n\nExhibit D -- Form of U.S. Tax ...,nnexhibit c form collateral agreementnnexhibit d form us tax compliance certificatenne...
9,secured_facility,Defined Terms.,defined terms


***

## Bulk Compare
Use One2Many to compare a large number of documents against a common base document.  For example, compare lease agreements against lease form to determine deviations from approved form. Use Many2Many to compare a large number of items (the 'Query') against another large groups of items (the 'Search') and determine matches. For example, a client requests we confirm the destruction of files relating to several hundred matters, where the matter description for each must be cross checked against several thousand entries in our file storage list. This tool leverages fuzzy matching in order to identify matches undetected by pure booleen searches (e.g., "FIRST INTERNATIONAL" captured even if search query is "INTL FIRS")

In [4]:
button_compare = widgets.Button(description="Submit")
button_compare.style.button_color = 'lightblue'

@button_compare.on_click
def on_click(b):
    doc_1_raw_sent_list = []
    doc_1_clean_sent_list = []
    sent_list = sent_tokenize(load_raw(file_path_text_1.value))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        doc_1_clean_sent_list.append(clean_sent)
        doc_1_raw_sent_list.append(sent)
    df_doc_1 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_1['clean'] = doc_1_clean_sent_list
    df_doc_1['raw'] = doc_1_raw_sent_list
    doc_2_raw_sent_list = []
    doc_2_clean_sent_list = []
    sent_list = sent_tokenize(load_raw(file_path_text_2.value))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        doc_2_clean_sent_list.append(clean_sent)
        doc_2_raw_sent_list.append(sent)
    df_doc_2 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_2['clean'] = doc_2_clean_sent_list
    df_doc_2['raw'] = doc_2_raw_sent_list
    df_results = pd.DataFrame(columns=['query', 'match', 'strength'])
    query = df_doc_1['clean']
    choices = df_doc_2['clean']
    query_list = []
    found_list = []
    strength_list = []
    for q in query:
        match = process.extractOne(q, choices=choices, scorer=fuzz.token_sort_ratio, score_cutoff=0)
        if match != None:
            index = match[2]
            if match[1] >= compare_slider.value:
                found = match[0]
                strength = match[1]
            else:
                found = 'None'
                strength = 'N/A'
            query_list.append(q)
            found_list.append(choices[index])
            strength_list.append(strength)
    df_results['query'] = query_list
    df_results['found']= found_list
    df_results['strength'] = strength_list
    display(df_results)

file_path_text_1 = widgets.Text(
    value='Stock Purchase Agreement_original.txt',
    placeholder='Type something',
    description='Query:',
    disabled=False
)

file_path_text_2 = widgets.Text(
    value='Stock Purchase Agreement_new_deal.txt',
    placeholder='Type something',
    description='Search:',
    disabled=False
)

s = widgets.Text(
    value='filename and path',
    placeholder='Type something',
    description='Base:',
    disabled=False
)

t = widgets.Text(
    value='filename and path',
    placeholder='Type something',
    description='Compare:',
    disabled=False
)

compare_slider = widgets.FloatSlider(
    value=0,
    base=100,
    min=0, # max exponent of base
    max=100, # min exponent of base
    step=0.2, # exponent step
    description='Similarity:'
)


tab1 = VBox(children=[HBox(children=[file_path_text_1, file_path_text_2, compare_slider])])
tab2 = VBox(children=[HBox(children=[s, t])])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Many2Many')
tab.set_title(1, 'One2Many')
VBox(children=[tab, button_compare])

***

## Find Precedent
Quickly search one or more curated clause banks for similar clauses.  Also, the user can identify form documents from which a particular agreememt originates.  The user can set a minimum similarity threshold to filter out less relevant clauses.  

In [23]:
button_compare = widgets.Button(description="Submit")
button_compare.style.button_color = 'lightblue'

@button_compare.on_click
def on_click(b):
    doc_1_raw_sent_list = []
    doc_1_clean_sent_list = []
    sent_list = sent_tokenize(load_raw(file_path_text_1.value))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        doc_1_clean_sent_list.append(clean_sent)
        doc_1_raw_sent_list.append(sent)
    df_doc_1 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_1['clean'] = doc_1_clean_sent_list
    df_doc_1['raw'] = doc_1_raw_sent_list
    doc_2_raw_sent_list = []
    doc_2_clean_sent_list = []
    sent_list = sent_tokenize(load_raw(file_path_text_2.value))
    for sent in sent_list:
        clean_sent = normalize_sent(sent)
        doc_2_clean_sent_list.append(clean_sent)
        doc_2_raw_sent_list.append(sent)
    df_doc_2 = pd.DataFrame(columns=['raw', 'clean'])
    df_doc_2['clean'] = doc_2_clean_sent_list
    df_doc_2['raw'] = doc_2_raw_sent_list
    df_results = pd.DataFrame(columns=['query', 'match', 'strength'])
    query = df_doc_1['clean']
    choices = df_doc_2['clean']
    query_list = []
    found_list = []
    strength_list = []
    for q in query:
        match = process.extractOne(q, choices=choices, scorer=fuzz.token_sort_ratio, score_cutoff=0)
        if match != None:
            index = match[2]
            if match[1] >= compare_slider.value:
                found = match[0]
                strength = match[1]
            else:
                found = 'None'
                strength = 'N/A'
            query_list.append(q)
            found_list.append(choices[index])
            strength_list.append(strength)
    df_results['query'] = query_list
    df_results['found']= found_list
    df_results['strength'] = strength_list
    display(df_results)

file_path_text_1 = widgets.Text(
    value='Stock Purchase Agreement_original.txt',
    placeholder='Type something',
    description='Query:',
    disabled=False
)

file_path_text_2 = widgets.Text(
    value='Stock Purchase Agreement_new_deal.txt',
    placeholder='Type something',
    description='Search:',
    disabled=False
)

number_returned = widgets.Dropdown(
    options=['1', '2', '3'],
    value='2',
    description='Number:',
    disabled=False,
)

t = widgets.Text(
    value='filename and path',
    placeholder='Type something',
    description='Compare:',
    disabled=False
)

compare_slider = widgets.FloatSlider(
    value=0,
    base=100,
    min=0, # max exponent of base
    max=100, # min exponent of base
    step=0.2, # exponent step
    description='Similarity:'
)


tab1 = VBox(children=[HBox(children=[file_path_text_1, file_path_text_2, compare_slider])])
tab2 = VBox(children=[HBox(children=[number_returned, t])])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Many2Many')
tab.set_title(1, 'One2Many')
VBox(children=[tab, button_compare])

Widget Javascript not detected.  It may not be installed or enabled properly.
Widget Javascript not detected.  It may not be installed or enabled properly.


## Clustering and Text Similarity Models
*Cluster text into similar topics.  Find similar text within several documents.*

In [8]:
!pip install bson

  Running setup.py bdist_wheel for bson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/b7/73/0d/3e4f9a49fb85347082f0a72575385c030047abd6aec14493fe
Successfully built bson
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


***

In [ ]:
import boto3
import json
import bson
#from bson import json_util
 
comprehend = boto3.client(service_name='comprehend')
                
input_s3_url = "s3://amazonaws.com/topichk/xyz.txt"
input_doc_format = "ONE_DOC_PER_LINE"
output_s3_url = "s3://amazonaws.com/blaze4/output.json"
data_access_role_arn = 'arn:aws:iam::701155529821:role/service-role/AmazonComprehendServiceRole-comprehend'
number_of_topics = 10
 
input_data_config = {"S3Uri": input_s3_url, "InputFormat": input_doc_format}
output_data_config = {"S3Uri": output_s3_url}
 
start_topics_detection_job_result = comprehend.start_topics_detection_job(NumberOfTopics=number_of_topics,
                                                                              InputDataConfig=input_data_config,
                                                                              OutputDataConfig=output_data_config,
                                                                              DataAccessRoleArn=data_access_role_arn)
 
print('start_topics_detection_job_result: ' + json.dumps(start_topics_detection_job_result))
 
job_id = start_topics_detection_job_result["JobId"]
 
print('job_id: ' + job_id)
 
describe_topics_detection_job_result = comprehend.describe_topics_detection_job(JobId=job_id)
 
print('describe_topics_detection_job_result: ' + json.dumps(describe_topics_detection_job_result, default=json_util.default))
 
list_topics_detection_jobs_result = comprehend.list_topics_detection_jobs()
 
print('list_topics_detection_jobs_result: ' + json.dumps(list_topics_detection_jobs_result, default=json_util.default))

## QuickCAN
Computer generated revisions to contracts based on precedent clause database.  Machine learning algorithms and fuzzy matching are used to incorporate revisions made to similar text in precedent clauses.

In [ ]:
x = widgets.RadioButtons(
    options=['finance', 'leasing', 'private equity', 'corporate M&A', 'bond financing'],
     value='finance',
    description='Type:',
    disabled=False
)
display(x)

IntSlider = widgets.SelectionSlider(
    options=['Aggressive', 'Favorable', 'Neutral', 'Must Haves'],
    value='Aggressive',
    description='Set Intensity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

display(IntSlider)

y = widgets.Text(
    value='filename and path',
    placeholder='Type something',
    description='Filename:',
    disabled=False
)

display(y)

from IPython.display import display
button = widgets.Button(description="Submit")
button.style.button_color = 'lightblue'
display(button)

def on_button_clicked(df_current, df_precedent):
    print('')
    #run_match(df_current, df_precedent)


button.on_click(on_button_clicked(y.value, y.value))

## QuickCAR

Computer generated responses to requests for production and interrogetories. Machine learning algorithms and fuzzy matching are used to incorporate historical responses to similar requests.  Will automatically generate a first draft of a response.

## Signature Page Generator
***
Generate signature page packets with the press of a button.

## Closing Binder Generator
***
Automate the preparation of closing binders.

In [23]:
@button.on_click
def on_click(b):
    pass

lender_widget = widgets.RadioButtons(
    options=['Wells Fargo', 'Bank of America (Private Wealth)', 'Bank of America', 'JPMorgan Chase', 'Ocean Bank', 'Other'],
    value='Ocean Bank',
    description='Lender:',
    disabled=False
)

toggle = widgets.RadioButtons(
    options=['Middle Market', 'Construction Loan', 'Bank Loan', 'Loan Modification', 'ABL Loan', 'Art Loan'],
    value='Bank Loan',
    description='Loan Type:',
    disabled=False
)

title_textbox = widgets.Text(
    value='RM',
    description='Title:',
)

tab1 = VBox(children=[HBox(children=[lender_widget, toggle])])
tab2 = VBox(children=[HBox(children=[title_textbox,])])
tab = widgets.Tab(children=[tab1, tab2])
tab.set_title(0, 'Loan Details')
tab.set_title(1, 'Contacts')
VBox(children=[tab, button])